# Monads for normal people!

This Jupyter notebook implements the code in the talk by Dustin Getz

> Video https://www.infoq.com/presentations/Monads-Code#

> Slides https://www.slideshare.net/eldariof/monads-in-python

References

> pymonad https://github.com/dustingetz/pymonads

> monadic-interpreter https://github.com/dustingetz/monadic-interpreter


# intended audience

* coders

* who are comfortable with lambdas

* who learn by example

# goals

* how monads work?
* how do monads help?
* are monads useful IRL?
* especially in enterprise?
* where do they fall short and what's next?

# large codebases are complex

* Spring, EJB, AspectJ, DI, AOP
* Common goal: make code look like business logic
* (to varying degrees of success)

# Aspect Oriented Programming

*From Wikipedia...*

# Lots of code to follow

* Pay attention to how functions compose

# a bank API

In [1]:
from collections import namedtuple

Person = namedtuple('Person', 'name')
Account = namedtuple('Account', 'id')
Balance = namedtuple('Balance', ['cash', 'ccy'])

alice = Person('Alice')
bob = Person('Bob')


def get_account(person):
    if person.name == 'Alice': return Account(1)
    elif person.name == 'Bob': return Account(2)
    else: return None

def get_balance(account):
    if account.id == 1: return Balance(1000000, 'usd')
    elif account.id == 2: return Balance(75000, 'usd')
    else: return None

def get_qualified_amount(balance):
    if balance.cash > 200000: return balance.cash
    else: return None


# what we want to write

In [2]:
def get_loan(name):
    account = get_account(name)
    balance = get_balance(account)
    loan = get_qualified_amount(balance)
    return loan

# business analyst would write this code

```python
# POSIX

alice | get_account | get_balance | get_qualified_amount
```

```python
# Object-Oriented

alice.get_account().get_balance().get_qualified_amount()
```

In [3]:
# Functions

get_qualified_amount( get_balance( get_account( alice ) ) )

1000000

# I love *AttributeErrors* !

In [4]:
# get_account(None)

# what the production code looks like :-(

In [5]:
def get_loan(name):
    account = get_balance(name)
    if not account:
        return None

    balance = get_balance(account)
    if not balance:
        return None

    loan = get_qualified_amount(balance)
    return loan

# factor! abstract! happy!

In [6]:
def bind(v, f):
    if v:            # v == alice
        return f(v)  # get_account(alice)
    else:
        return None


In [7]:
alice = Person('Alice')
bind(alice, get_account)

Account(id=1)

In [8]:
def bind(v, f):
    if v:
        return f(v)
    else:            # v == None
        return None  # don't call f


In [9]:
alice = Person(None)
print(bind(None, get_account))

None


# the code we *really* want to write

In [10]:
def bind(v, f): return f(v) if v else None

def get_loan(name):
    account = bind(name, get_account)
    balance = bind(account, get_balance)
    loan = bind(balance, get_qualified_amount)
    return loan

In [11]:
alice = Person('Alice')

get_loan(alice)

1000000

In [12]:
get_loan(None)

# or more succinctly

In [13]:
def bind(v, f): return f(v) if v else None

def m_pipe(val, fns):
    m_val = val
    for f in fns:
        m_val = bind(m_val, f)
    return m_val

fns = [get_account, get_balance, get_qualified_amount]

m_pipe(alice, fns)

1000000

In [14]:
dustin = Person('Dustin')

m_pipe(dustin, fns)

# big picture goal

* make the code look like the business logic

* *"good closure programmers write a language to write their programs in"* -- DSL

* build a language to build your business logic

* add features without changing your business logic

![image.png](attachment:image.png)

* *"Great things are made of little things"* - Chinese proverb

# add a feature to our API

In [15]:
from collections import namedtuple

Person = namedtuple('Person', 'name')
Account = namedtuple('Account', 'id')
Balance = namedtuple('Balance', ['cash', 'ccy'])

# Add Error Handling festure to API
ValueWithError = namedtuple('ValueWithError', ['value', 'error'])

alice = Person('Alice')
bob = Person('Bob')


def get_account(person):
    if person.name == 'Alice': return ValueWithError(Account(1), None)
    elif person.name == 'Bob': return ValueWithError(Account(2), None)
    else: return ValueWithError(None, "No account for {}".format(person.name))

def get_balance(account):
    if account.id == 1: return ValueWithError(Balance(1000000, 'usd'), None)
    elif account.id == 2: return ValueWithError(Balance(75000, 'usd'), None)
    else: return ValueWithError(None, "No balance for account {}".format(account.id))

def get_qualified_amount(balance):
    if balance.cash > 200000: return ValueWithError(balance, None)
    else: return ValueWithError(None, "Insufficient funds of {}".format(balance.cash))



# what does *bind* look like now ?

In [16]:
def bind(mval, mf):       # mval == (ValueWithError(Account(1), None)
    value = mval.value    # value == Account(1)
    error = mval.error    # error == None
    if not error:
        return mf(value)  # mf(Account(1))
    else:                 # mval == (ValueWithError(None, "Insufficient funds !!")
        return mval       # don't call mf


In [18]:
mval = ValueWithError(Account(1), None)

bind(mval, get_balance)

ValueWithError(value=Balance(cash=1000000, ccy='usd'), error=None)

In [19]:
mval = ValueWithError(None, "Insufficient funds !!")

bind(mval, get_balance)

ValueWithError(value=None, error='Insufficient funds !!')

In [ ]:
# Slide 20/47 